In [8]:
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer

In [19]:
class WafDetector:
    def __init__(self):
        self.predictor = joblib.load('./request_predictor.joblib')

    def parse_payload(self, payload):
        """
        Parse and format the payload for prediction.
        The payload should be converted into a list of strings.
        """
        if isinstance(payload, str):
            return [payload]
        elif isinstance(payload, list):
            return payload
        else:
            raise ValueError("Payload must be a string or a list of strings.")

    def predict(self, payload):
        """
        Predict the type of the payload.
        """
        parsed_payload = self.parse_payload(payload)
        print(parsed_payload)
        return self.predictor.predict(parsed_payload)

In [20]:
waf = WafDetector()

In [26]:
result = waf.predict("""username=kiannaquines&password=password@0000&is_admin=true&submit=true' OR '1' = '1 --""")

["username=kiannaquines&password=password@0000&is_admin=true&submit=true' OR '1' = '1 --"]


In [27]:
print("Prediction:", result)

Prediction: ['sqli']
